<a href="https://colab.research.google.com/github/rajeshradhakrishnanmvk/moonlight/blob/main/Agent_Development_Kit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-adk

In [2]:
import asyncio
from datetime import datetime, timedelta

In [3]:
# ------------------------------
# 1. Application Registry
# ------------------------------
# This registry keeps track of all created applications.
class AppRegistry:
    def __init__(self):
        self.apps = {}

    def get_app(self, app_id: str):
        return self.apps.get(app_id)

    def save_app(self, app_id: str, app_data: dict):
        self.apps[app_id] = app_data

    def delete_app(self, app_id: str):
        if app_id in self.apps:
            del self.apps[app_id]

In [4]:
# Global registry instance
app_registry = AppRegistry()

In [5]:
# ------------------------------
# 2. Simulated LLM Agent (LLMAgent)
# ------------------------------
# In a production ADK solution you would use an ADK Agent (or LiteLLM wrapper)
# to interact with a large language model. Here we simulate the API call.
class LLMAgent:
    async def generate_completion(self, prompt: str) -> str:
        # Simulate delay and simple code generation based on the prompt.
        await asyncio.sleep(0.5)
        if "calculator" in prompt.lower():
            # A simple code snippet for a basic calculator
            return (
                "def calculator(a, b):\n"
                "    return {\n"
                "       'add': a + b,\n"
                "       'subtract': a - b,\n"
                "       'multiply': a * b,\n"
                "       'divide': a / b if b != 0 else None\n"
                "    }"
            )
        elif "upgrade" in prompt.lower():
            # A simulated upgrade that adds a scientific function
            return (
                "def scientific_calculator(a, b):\n"
                "    import math\n"
                "    result = calculator(a, b)\n"
                "    result['power'] = a ** b\n"
                "    result['sqrt_a'] = math.sqrt(a) if a >= 0 else None\n"
                "    return result"
            )
        else:
            return "Simulated output for prompt: " + prompt

In [6]:
# ------------------------------
# 3. Agent Classes
# ------------------------------

# CreatorAgent creates a new app (if it does not exist) and adds a feature using the LLM.
class CreatorAgent:
    def __init__(self, llm_agent: LLMAgent):
        self.llm_agent = llm_agent

    def create_app(self, app_id: str) -> dict:
        print(f"CreatorAgent: Creating new app '{app_id}'")
        new_app = {
            "id": app_id,
            "features": {},
            "createdAt": datetime.now(),
            "updatedAt": datetime.now()
        }
        app_registry.save_app(app_id, new_app)
        return new_app

    async def add_feature_with_llm(self, app: dict, feature_name: str, feature_prompt: str) -> dict:
        print(f"CreatorAgent: Adding feature '{feature_name}' using LLM (prompt: {feature_prompt})")
        generated_code = await self.llm_agent.generate_completion(feature_prompt)
        app["features"][feature_name] = {
            "generatedCode": generated_code,
            "info": "Feature generated by LLMAgent"
        }
        app["updatedAt"] = datetime.now()
        app_registry.save_app(app["id"], app)
        return app

In [7]:
# MaintainerAgent upgrades an existing feature.
class MaintainerAgent:
    def __init__(self, llm_agent: LLMAgent):
        self.llm_agent = llm_agent

    def update_feature(self, app: dict, feature_name: str, additional_data: dict) -> dict:
        print(f"MaintainerAgent: Updating feature '{feature_name}' for app '{app['id']}'")
        if feature_name in app["features"]:
            app["features"][feature_name].update(additional_data)
            app["updatedAt"] = datetime.now()
            app_registry.save_app(app["id"], app)
        return app

    async def update_feature_with_llm(self, app: dict, feature_name: str, upgrade_prompt: str) -> dict:
        print(f"MaintainerAgent: Upgrading feature '{feature_name}' using LLM (prompt: {upgrade_prompt})")
        generated_upgrade = await self.llm_agent.generate_completion(upgrade_prompt)
        if feature_name in app["features"]:
            app["features"][feature_name]["upgraded"] = generated_upgrade
            app["updatedAt"] = datetime.now()
            app_registry.save_app(app["id"], app)
        else:
            print(f"MaintainerAgent: Feature '{feature_name}' does not exist. Cannot upgrade.")
        return app

In [8]:
# DestroyerAgent cleans up stale applications from the registry.
class DestroyerAgent:
    def __init__(self, max_age_seconds: int = 300):
        self.max_age = timedelta(seconds=max_age_seconds)

    def check_and_destroy_stale_apps(self):
        print("DestroyerAgent: Checking for stale apps...")
        now = datetime.now()
        stale_apps = [
            app_id for app_id, app in app_registry.apps.items()
            if now - app["updatedAt"] > self.max_age
        ]
        for app_id in stale_apps:
            print(f"DestroyerAgent: App '{app_id}' is stale. Removing it.")
            app_registry.delete_app(app_id)

In [9]:
# CouncilController coordinates the agents.
class CouncilController:
    def __init__(self):
        self.llm_agent = LLMAgent()
        self.creator = CreatorAgent(self.llm_agent)
        self.maintainer = MaintainerAgent(self.llm_agent)
        self.destroyer = DestroyerAgent()

    async def process_request(
        self,
        app_id: str,
        feature_name: str = None,
        feature_prompt: str = None,
        fallback_data: dict = None
    ) -> dict:
        # Retrieve or create the app.
        app = app_registry.get_app(app_id)
        if not app:
            app = self.creator.create_app(app_id)

        # If a feature name is provided, either add or update it.
        if feature_name:
            if feature_name not in app["features"]:
                if feature_prompt:
                    app = await self.creator.add_feature_with_llm(app, feature_name, feature_prompt)
                else:
                    app["features"][feature_name] = fallback_data or {"created": True}
                    app["updatedAt"] = datetime.now()
                    app_registry.save_app(app_id, app)
            else:
                if feature_prompt:
                    app = await self.maintainer.update_feature_with_llm(app, feature_name, feature_prompt)
                else:
                    app = self.maintainer.update_feature(app, feature_name, fallback_data or {"updated": True})
        # Clean up stale apps after processing.
        self.destroyer.check_and_destroy_stale_apps()
        return app

In [11]:
# Initialize the CouncilController to coordinate the agents.
council = CouncilController()

In [13]:
# Simulate a new app request to create a calculator application.
app_id = "calc"
feature_name = "calculator"
feature_prompt = ("Create a simple calculator web app with addition, subtraction, "
                  "multiplication, and division.")

In [14]:
print("\n=== Processing App Creation Request ===")
app = await council.process_request(app_id, feature_name, feature_prompt, {"info": "Initial feature data"})
print("\n--- App after creation ---")
print(app)
print("\n--- Current Registry ---")
print(app_registry.apps)


=== Processing App Creation Request ===
MaintainerAgent: Upgrading feature 'calculator' using LLM (prompt: Create a simple calculator web app with addition, subtraction, multiplication, and division.)
DestroyerAgent: Checking for stale apps...

--- App after creation ---
{'id': 'calc', 'features': {'calculator': {'generatedCode': "def calculator(a, b):\n    return {\n       'add': a + b,\n       'subtract': a - b,\n       'multiply': a * b,\n       'divide': a / b if b != 0 else None\n    }", 'info': 'Feature generated by LLMAgent', 'upgraded': "def calculator(a, b):\n    return {\n       'add': a + b,\n       'subtract': a - b,\n       'multiply': a * b,\n       'divide': a / b if b != 0 else None\n    }"}}, 'createdAt': datetime.datetime(2025, 4, 10, 8, 10, 39, 801417), 'updatedAt': datetime.datetime(2025, 4, 10, 8, 12, 3, 692531)}

--- Current Registry ---
{'calc': {'id': 'calc', 'features': {'calculator': {'generatedCode': "def calculator(a, b):\n    return {\n       'add': a + b,

In [15]:
# Simulate an upgrade request for the same feature.
upgrade_prompt = ("Upgrade the calculator app to include scientific functions such as square root "
                  "and exponentiation.")
print("\n=== Processing App Upgrade Request ===")
app = await council.process_request(app_id, feature_name, upgrade_prompt, {"info": "Upgraded feature data"})
print("\n--- App after upgrade ---")
print(app)
print("\n--- Final Registry ---")
print(app_registry.apps)


=== Processing App Upgrade Request ===
MaintainerAgent: Upgrading feature 'calculator' using LLM (prompt: Upgrade the calculator app to include scientific functions such as square root and exponentiation.)
DestroyerAgent: Checking for stale apps...

--- App after upgrade ---
{'id': 'calc', 'features': {'calculator': {'generatedCode': "def calculator(a, b):\n    return {\n       'add': a + b,\n       'subtract': a - b,\n       'multiply': a * b,\n       'divide': a / b if b != 0 else None\n    }", 'info': 'Feature generated by LLMAgent', 'upgraded': "def calculator(a, b):\n    return {\n       'add': a + b,\n       'subtract': a - b,\n       'multiply': a * b,\n       'divide': a / b if b != 0 else None\n    }"}}, 'createdAt': datetime.datetime(2025, 4, 10, 8, 10, 39, 801417), 'updatedAt': datetime.datetime(2025, 4, 10, 8, 15, 13, 529905)}

--- Final Registry ---
{'calc': {'id': 'calc', 'features': {'calculator': {'generatedCode': "def calculator(a, b):\n    return {\n       'add': a +